In [32]:
from ib_insync import *
util.startLoop()


ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)


<IB connected to 127.0.0.1:7497 clientId=2>

In [141]:
order_records = {}

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

portfolio = {"USD":0, "HKD": 0, "AUD": 0, "JPY": 0}
for key in country_info:
    for p in country_info[key]["Pairs"]:
        adr = p[0]+"_adr"
        portfolio[adr] = 0
        
        under = p[1] + "_stk"
        portfolio[under] = 0
        
        

class my_order:
    def __init__(self, sec_name, order_type, sec_type, total_quantity):
        
        self.sec_name = sec_name
        self.sec_type = sec_type # if foreign currency, put ""
        self.order_type = order_type # Either "buy" or "sell"
        self.status = "Pending"
        self.total_quantity = total_quantity

        self.filled_quantity = 0
        self.avg_filled_price = 0.0

def reset_portfolio():
    global portfolio:
        
    for key in country_info:
        for p in country_info[key]["Pairs"]:
            adr = p[0]+"_adr"
            portfolio[adr] = 0

            under = p[1] + "_stk"
            portfolio[under] = 0

        

def onOrderStatus(trade): 
    global order_records, portfolio
    print(f"Update in order {trade.order.orderId}")
#     print(trade.orderStatus.status == 'Cancelled')
    #print('order status' + trade.orderStatus.status)
    orderId = trade.order.orderId
    if trade.orderStatus.status == 'Cancelled':
        order_records[orderId].status = "Cancelled"
        print("Order has been cancelled.")
    elif trade.orderStatus.status == "Filled":
        
        change = trade.orderStatus.filled - order_records[orderId].filled_quantity
        order_records[orderId].filled_quantity = trade.orderStatus.filled
        order_records[orderId].avg_filled_price = trade.orderStatus.avgFillPrice
        
        sec_type = order_records[orderId].sec_type
        sec =  order_records[orderId].sec_name
        order_type = order_records[orderId].order_type
        if sec_type == 'adr':
            if order_records[orderId].order_type == "buy":
                portfolio[sec + sec_type] += change
            else:
                portfolio[sec + sec_type] -= change
                
        elif sec_type == "underlying":
            if order_records[orderId].order_type == "buy":
                portfolio[sec + '_stk'] += change
            else:
                portfolio[sec +  '_stk'] -= change
        
        
        print(f"{trade.orderStatus.filled} shares of {order_type} {sec} ({sec_type}) has been filled at an average price of {trade.orderStatus.avgFillPrice}")
        if rade.orderStatus.remaining == 0:
            order_records[orderId].status = "Completed"
            print("Order is completed!")
            
    else:
        print(trade.orderStatus.status)
    
    
    

def onExecDetails(trade,fill): print('order exec details' + fill.execution)


ib.execDetailsEvent+=onExecDetails 
ib.orderStatusEvent+=onOrderStatus

# fut = Future('ES', '20181221', 'GLOBEX') 
# ib.qualifyContracts(fut) 

contract = Stock('TSLA', 'SMART', 'USD')
ib.qualifyContracts(contract)


order =LimitOrder('BUY', 1,1) 
trade = ib.placeOrder(contract, order) 
print(trade)
order_records[trade.order.orderId] = my_order("AAPL","Stock","buy",1)

ib.run()

Trade(contract=Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), order=LimitOrder(orderId=63, clientId=2, action='BUY', totalQuantity=1, lmtPrice=1), orderStatus=OrderStatus(orderId=63, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2021, 4, 23, 1, 53, 10, 933879, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])
Update in 63
Update in order 63
trade.orderStatus.status
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted
Update in order 63
PreSubmitted


In [49]:
order_records[46]

In [142]:
import os
from datetime import datetime
def place_order(country, security, sectype, side, quantity, price, market_order = False):
    global order_books
    exchange = country_info[country]["Exchange"]
    currenct = country_info[country]["Currency"]
    
    # Define order 
    contract = Contract(symbol = security, secType = 'STK', exchange =exchange, currency = currency)
    if market_order == True:
        order =  MarketOrder(side,quantity)
    else:
        order = LimitOrder(side,quantity,price)
    
    # Place the order
    msg = ib.placeOrder(contract, order)
    orderId = msg.order.orderId
    
    # Add the orderId to orderbook
    name =  security +"_"+ sectype
    order_books[name].add(orderId)
    order_records[orderId] = my_order(security, side, sectype, quantity)
    

def print_order_records():
    global order_records
    
    #sys.__stdout__.write("orderId\tSecurity\tSecType\tOrderType\tStatus\tTotal\tFilled\tAvgPrice\n")
    print(datetime.now())
    print("orderId\tSecurity\tSecType\tOrderType\tStatus\tTotal\tFilled\tAvgPrice\n")
    for order in order_records:
        orderId = order
        Security = order_records[order].sec_name
        Type = order_records[order].sec_type
        Status = order_records[order].status
        Total = order_records[order].total_quantity
        Filled = order_records[order].filled_quantity
        AvgPrice = order_records[order].avg_filled_price
        order_type = order_records[order].order_type
        sys.__stdout__.write(f"{orderId}\t{Security}\t\t{Type}\t{order_type}\t\t{Status}\t{Total}\t{Filled}\t{AvgPrice}\n")


def print_portfolio():
    global portfolio
    #sys.__stdout__.write("Security\tPosition\n")
    print(datetime.now())
    print("Security\tPosition\n")
    for key in portfolio:
        
        if len(key) <= 7:
            #sys.__stdout__.write(f"{key}\t\t{portfolio[key]}\n")
            print(f"{key}\t\t{portfolio[key]}\n")
        else:
            print(f"{key}\t{portfolio[key]}\n")
            #sys.__stdout__.write(f"{key}\t{portfolio[key]}\n")
            

    
    
    


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; jfarm; eufarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; jfarm; eufarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.


In [147]:
def liquidate():
    """
    Func to liquidate all our current positions
    
    """
    
    global portfolio
    positions = ib.positions()  # A list of positions, according to IB
    for position in positions:
        action = ""
        contract = position.contract
        if position.position > 0: # Number of active Long positions
            action = 'Sell' # to offset the long positions
        elif position.position < 0: # Number of active Short positions
            action = 'Buy' # to offset the short positions
        else:
            assert False
        totalQuantity = abs(position.position)
        order = MarketOrder(action=action, totalQuantity=totalQuantity)
        trade = ib.placeOrder(contract, order)
        orderId = trade.orderId
        security = trade.contract.symbol
        sectype = ""
        if trade.contract.currency == "USD": # if usd, we know it is an adr
            sectype = "adr"
        else:
            sectype = "stk"
            

            # Add the orderId to orderbook
        name =  security +"_"+ sectype
        order_books[name].add(orderId)
        order_records[orderId] = my_order(security, action, sectype, position)


        print(f'Flatten Position: {action} {totalQuantity} {contract.localSymbol}')
    reset_portfolio()

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has bee

In [138]:
print_order_records()

2021-04-22 20:59:38.518717
orderId	Security	SecType	OrderType	Status	Total	Filled	AvgPrice



Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; jfarm; eufarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; jfarm; eufarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
